In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from utils import three_d_resnet_bi_lstm
from utils import datagenerator
from utils import metric
from utils import sequence_modeling
from utils import datasplit

import glob

In [2]:
# to specify a path to demographics data of the speakers of the CREMA-D corpus
id_phrases, id_sp_train, id_sp_valid, id_sp_test = datasplit.get_id_sp('VideoDemographics.csv')

In [3]:
len_seq = 60 # sequence length
size = 88 # size image
chan = 3 # number of image channel
pad = True # to use padding
bs = 4 # bach size
n_cl = 12 # number of classes
emo = 'FEA' # emotion choice
name_model = '3D'

path_metadata = 'D:/Databases/CREMA-D lips reading/'

p_im_all = glob.glob(path_metadata + 'images/*/*.jpg')
df_all_path = sequence_modeling.get_data(p_im_all, id_phrases)
df_train_em = df_all_path[(df_all_path.id_speaker.isin(id_sp_train))&(df_all_path.emotion==emo)]
p_train, n_train, phr_train, l_train = sequence_modeling.get_sequence(df_train_em, len_seq=len_seq, c_fr=False)
p_val, n_val, phr_val, l_val = sequence_modeling.get_sequence(df_all_path[df_all_path.id_speaker.isin(id_sp_valid)], len_seq=len_seq, c_fr=False)
p_test, n_test, phr_test, l_test = sequence_modeling.get_sequence(df_all_path[df_all_path.id_speaker.isin(id_sp_test)], len_seq=len_seq, c_fr=False)

100%|█████████████████████████████████████████████████████████████████████████████| 1558/1558 [00:08<00:00, 181.90it/s]


In [4]:
train_generator = datagenerator.DataGeneratorTrain(p_train, l_train, shape = (size, size), bs=10, shuffle=True, p_a=0, chan=chan, pad=pad)
valid_generator = datagenerator.DataGeneratorTest(p_val, l_val, n_val, shape = (size, size), bs=10, shuffle=False, chan=chan, pad=pad)
test_generator = datagenerator.DataGeneratorTest(p_test, l_test, n_test, shape = (size, size), bs=10, shuffle=False, chan=chan, pad=pad)

INIT_LR = 0.0001
EPOCHS = 10

name_app = '3d_lstm_ls_{}_size_{}_chan_{}_pad_{}_emo_{}_bs_{}_lr_{}_eps_{}'.format(len_seq, size, chan, pad, emo, bs, str(INIT_LR).split('.')[1], EPOCHS)

opt = tf.keras.optimizers.Adam(learning_rate=INIT_LR)

model = three_d_resnet_bi_lstm.build_three_d_resnet_18((len_seq, size, size, chan), n_cl, 'softmax', None,True, name_model)

metrics = metric.Metrics(path_save_weight = 'models/', verbose=1, patience=5, name_app=name_app)

callbacks = [
             metrics,
            ]

model.valid_generator = valid_generator
model.test_generator = test_generator

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

model.fit(
    train_generator,
    epochs = EPOCHS,
    verbose = True,
    shuffle = True, 
    callbacks = [callbacks],
    workers=4,
    )

models/2022-10-26_18-14-45_3d_lstm_ls_60_size_88_chan_3_pad_True_emo_FEA_bs_4_lr_0001_eps_10
Epoch 1/10
263/263 [==============================] - 179s 639ms/step - loss: 4.4434 - accuracy: 0.1678


100%|████████████████████████████████████████████████████████████████████████████████| 467/467 [47:10<00:00,  6.06s/it]


Recall improved from 0 to 0.08333333333333333
max_val_recall 0.08333333333333333

current_val_recall 0.08333333333333333

Epoch 2/10
263/263 [==============================] - 188s 708ms/step - loss: 3.3423 - accuracy: 0.2187


100%|████████████████████████████████████████████████████████████████████████████████| 467/467 [41:50<00:00,  5.38s/it]


Recall improved from 0.08333333333333333 to 0.11795364546936875
max_val_recall 0.11795364546936875

current_val_recall 0.11795364546936875

Epoch 3/10
263/263 [==============================] - 168s 635ms/step - loss: 2.6432 - accuracy: 0.3006


...

Recall did not improve.
max_val_recall 0.5011719659911483

current_val_recall 0.4752557271228655

Epoch 9/10
263/263 [==============================] - 168s 635ms/step - loss: 0.7311 - accuracy: 0.7621


100%|████████████████████████████████████████████████████████████████████████████████| 467/467 [41:58<00:00,  5.39s/it]


Recall improved from 0.5011719659911483 to 0.5227351883208803
max_val_recall 0.5227351883208803

current_val_recall 0.5227351883208803

Epoch 10/10
263/263 [==============================] - 170s 644ms/step - loss: 0.6254 - accuracy: 0.7912


100%|████████████████████████████████████████████████████████████████████████████████| 467/467 [44:23<00:00,  5.70s/it]

Recall did not improve.
max_val_recall 0.5227351883208803

current_val_recall 0.3600589734630615

